In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

##  Regulator Design: observer and state feedback

This example demonstrates the design of a regulator: a type of controller composed by an observer and a full-state feedback controller. It can be proven that, thanks to the separation principle, the controller and the observer can be designed independently - closed-loop eigenvalues and observer dynamics can be set separately without affecting each other.

Nonetheless, closed-loop transient performance depends on how fast the observer is with respect to the desired closed-loop dynamics. 

This example shows the design of a regulator for the controllable and observable system:

\begin{cases}
\dot{\textbf{x}}=\begin{bmatrix}1&0&3\\0&-4&-1\\0&1&-4\end{bmatrix}\textbf{x}+\begin{bmatrix}0\\0\\1\end{bmatrix}\textbf{u} \\ \\
\textbf{y}=\begin{bmatrix}1&0&0\end{bmatrix}\textbf{x}
\end{cases}

that has the transfer function:

$$
G(s) = C(sI-A)^{-1}B.
$$

### Development of the state feedback
The goal is to place 3 eigenvalues in $-1$ rad/s or faster in order to have a good transient response. A possible solution is: $K = \begin{bmatrix}\frac{8}{15}&-4.4&-4\end{bmatrix}$.


### Development of the observer
Given the eigenvalues of the controlled system, a better (faster and stable) choice for the observer is $\lambda_{1,2,3} = -10$ rad/s. This can be achieved with $L=\begin{bmatrix}23&66&\frac{107}{3}\end{bmatrix}^T$.


### Composition of the regulator

The regulator can be implemented in two ways, as a transfer function:

<img src="Images\Block_regulator.png">

where:

$$
K(s) = -K(sI-A+LC+BK)^{-1}L\,.
$$

or as an observer with static feedback:

<img src="Images\Block_regulator2.png">

Recall that although the obtained closed dynamics is the same, the transfer function implementation may lead to an unstable controller - the eigenvalues of matrix (matrix $A-BK-LC$) may be unstable even if closed-loop dynamics is stable. 

### How to use this notebook?
- Try to change the initial conditions of the estimator (default is $\begin{bmatrix}0.2&0.2&0.2\end{bmatrix}^T$) and the observer's eigenvalues and see how the controlled system behaviour changes.
- Try to change the values in order to achieve settling time for 5% tolerance band of less than 2 s.

**Note:** 

- The ideal values refer to the case in which all the states of the system can be measured.
- The **Inverse reference gain** slider denotes the value for which the reference is divided (as you change the static gain of the closed-loop transfer function); in order to reach zero error for the step response its value is equal to the static gain.

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix('1 0 3; 0 -4 -1; 0 1 -4')
B = numpy.matrix('0; 0; 1')
C = numpy.matrix('1 0 0')
X0 = numpy.matrix('0.2; 0.2; 0.2')
K = numpy.matrix([8/15,-4.4,-4])
L = numpy.matrix([[23],[66],[107/3]])

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(3,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-1.])) 
eig2c.setM(numpy.matrix([[-1.],[0.]]))
eig3c.setM(numpy.matrix([-1.]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig1o.setM(numpy.matrix([-10.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))
eig3o.setM(numpy.matrix([-10.]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Set K and L', 'Set the eigenvalues'],
    value= 'Set the eigenvalues',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 complex eigenvalues', '2 complex eigenvalues'],
    value= '0 complex eigenvalues',
    description='Complex eigenvalues:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulse', 'step', 'sinusoid', 'square wave'],
    value='step',
    description='Type of reference:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description='Reference:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.01,
    max=4,
    step=0.01,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

gain_id_w = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

gain_w2 = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

gain_id_w2 = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

In [5]:
# Support functions

def eigen_choice(sele):
    if sele == '0 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 complex eigenvalues':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Set K and L':
        method = 1
        sele.disabled = True
    if selm == 'Set the eigenvalues':
        method = 2
        sele.disabled = False
    return method

## Implementation as transfer function controller

In [6]:
sols = numpy.linalg.eig(A)

def main_callback(Aw, Bw, X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, eig3o, u, period, selm, sele, selu, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig3o[0,0], 
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    Gs = sss(A,B,C,0)
    Ks = sss(A-B*K-L*C,L,-K,0)
    Fs = control.series(-Ks,Gs)
    sys = control.feedback(Fs)
    
    Gs_id = sss(A,B,sym.eye(3),sym.zeros(3,1))
    Fs_id = control.series(K,Gs_id)
    A1 = numpy.matrix(Fs_id.A-Fs_id.B*Fs_id.C)
    B1 = numpy.matrix(Fs_id.B*sym.Matrix([[1],[0],[0]]))
    C1 = numpy.matrix(sym.Matrix([1,0,0]).T*Fs_id.C)
    sys_id = sss(A1,B1,C1,0)
    
    sys_o = sss(A-L*C,numpy.hstack((L,B)),sym.eye(3),sym.zeros(3,2))
    
    dcgain = control.dcgain(sys)
    t = numpy.linspace(0, 1000, 2)
    t, y = control.step_response(sys_id,t)
    dcgain_id = y[-1]
    gain_w.value = dcgain
    gain_id_w.value = dcgain_id
    if dcgain != 0 and dcgain_id != 0:
        u1 = u/gain_w.value
        u2 = u/gain_id_w.value
    else:
        print('The inverse gain set is 0 and it is changed to 1')
        u1 = u/1
        u2 = u/1
    
    solc = numpy.linalg.eig(sys.A)
    solo = numpy.linalg.eig(A-L*C-B*K)
    print('The system\'s eigenvalues are:', round(sols[0][0],2),',', round(sols[0][1],2),'and', round(sols[0][2],2))
    print('The controlled closed loop system\'s eigenvalues are:', 
          round(solc[0][0],2),',', 
          round(solc[0][1],2),',', 
          round(solc[0][2],2),',',
          round(solc[0][3],2),',',
          round(solc[0][4],2),'and',
          round(solc[0][5],2))
    print('The controller\'s eigenvalues are:', round(solo[0][0],2),',', round(solo[0][1],2),'and', round(solo[0][2],2))
    print('')
    print('The static gain of the closed loop system (from the reference to the output) is: %.5f' %dcgain)
    print('The static gain of the closed loop ideal system (from the reference to the output) is: %.5f' %dcgain_id)
    
    X0w1 = numpy.matrix([[X0w[0,0]],[X0w[1,0]],[X0w[2,0]],[0],[0],[0]])
    T = numpy.linspace(0, 12, 1000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        U2 = [0 for t in range(0,len(T))]
        U2[0] = u2
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
        T, yout_k, xout_k = control.forced_response(Ks,T,yout-U1,X0w)
        T, yout_o, xout_o = control.forced_response(sys_o,T,[yout,yout_k],X0w)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        U2 = [u2 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
        T, yout_k, xout_k = control.forced_response(Ks,T,yout-U1,X0w)
        T, yout_o, xout_o = control.forced_response(sys_o,T,[yout,yout_k],X0w)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        U2 = u2*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
        T, yout_k, xout_k = control.forced_response(Ks,T,yout-U1,X0w)
        T, yout_o, xout_o = control.forced_response(sys_o,T,[yout,yout_k],X0w)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U2 = u2*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
        T, yout_k, xout_k = control.forced_response(Ks,T,yout-U1,X0w)
        T, yout_o, xout_o = control.forced_response(sys_o,T,[yout,yout_k],X0w)
    # N.B. i primi 3 stati di xout sono quelli dello stimatore, mentre gli ultimi 3 sono quelli del sistema "reale"
    
    fig = plt.figure(num='Simulation1', figsize=(16,17))
    mag, phase, omega = control.bode_plot(sys,Plot = False)
    mag = control.mag2db(mag)
    phase = phase*180/numpy.pi
    fig.add_subplot(321)
    plt.title('Bode plot: magnitude')
    plt.semilogx(omega,mag)
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('Mag. [dB]')
    plt.grid(True,which="both")
    
    fig.add_subplot(323)
    plt.title('Bode plot: phase')
    plt.semilogx(omega,phase)
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('Phase [deg]')
    plt.grid(True,which="both")
    
    fig.add_subplot(325)
    plt.title('Output response')
    plt.ylabel('Output')
    plt.plot(T,yout,T,yout_id,'g',T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$y$','$y_{ideal}$','Reference'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(322)
    plt.title('First state response')
    plt.ylabel('$x_1$')
    plt.plot(T,xout_o[0],T,xout[3],T,xout_id[0],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{1est}$','$x_{1real}$','$x_{1ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(324)
    plt.title('Second state response')
    plt.ylabel('$x_2$')
    plt.plot(T,xout_o[1],T,xout[4],T,xout_id[1],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{2est}$','$x_{2real}$','$x_{2ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(326)
    plt.title('Third state response')
    plt.ylabel('$x_3$')
    plt.plot(T,xout_o[2],T,xout[5],T,xout_id[2],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{3est}$','$x_{3real}$','$x_{3ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()

   
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Eigenvalues:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Eigenvalues:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          eig3o,
                                          widgets.Label(' ',border=3),
                                          widgets.VBox([widgets.Label('Inverse reference gain:',border=3),
                                                        widgets.Label('Inverse ideal reference gain:',border=3)]),
                                          widgets.VBox([gain_w,gain_id_w])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START])])
out = widgets.interactive_output(main_callback, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'DW':DW})
out.layout.height = '1120px'
display(out, alltogether)

Output(layout=Layout(height='1120px'))

## Implementation as observer

In [7]:
sols = numpy.linalg.eig(A)

def main_callback2(Aw, Bw, X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, eig3o, u, period, selm, sele, selu, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig3o[0,0], 
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    Gs = sss(A,B,numpy.vstack((numpy.eye(3),[0,0,0])),[[0],[0],[0],[1]])
    Os = sss(A-L*C,numpy.hstack((L,B)),numpy.vstack((-K,numpy.eye(3))),[[0,0],[0,0],[0,0],[0,0]])
    Gas = control.append(Gs,Os)
    sys = control.connect(Gas,[[2,1],[3,4],[1,5]],[1],[1,2,3,6,7,8])
    
    Gs_id = sss(A,B,sym.eye(3),sym.zeros(3,1))
    Fs_id = control.series(K,Gs_id)
    A1 = numpy.matrix(Fs_id.A-Fs_id.B*Fs_id.C)
    B1 = numpy.matrix(Fs_id.B*sym.Matrix([[1],[0],[0]]))
    C1 = numpy.matrix(sym.Matrix([1,0,0]).T*Fs_id.C)
    sys_id = sss(A1,B1,C1,0)

    
    dcgain = control.dcgain(sys[0,0])
    t = numpy.linspace(0, 1000, 2)
    t, y = control.step_response(sys_id,t)
    dcgain_id = y[-1]
    gain_w2.value = dcgain
    gain_id_w2.value = dcgain_id
    if dcgain != 0 and dcgain_id != 0:
        u1 = u/gain_w2.value
        u2 = u/gain_id_w2.value
    else:
        print('The inverse gain setted is 0 and it is changed to 1')
        u1 = u/1
        u2 = u/1
    
    solc = numpy.linalg.eig(sys.A)
    print('The system\'s eigenvalues are:', round(sols[0][0],2),',', round(sols[0][1],2),'and', round(sols[0][2],2))
    print('The controlled closed loop system\'s eigenvalues are:', 
          round(solc[0][0],2),',', 
          round(solc[0][1],2),',', 
          round(solc[0][2],2),',',
          round(solc[0][3],2),',',
          round(solc[0][4],2),'and',
          round(solc[0][5],2))
    print('')
    print('The static gain of the closed loop system (from the reference to the output) is: %.5f' %dcgain)
    print('The static gain of the closed loop ideal system (from the reference to the output) is: %.5f' %dcgain_id)
    
    X0w1 = numpy.matrix([[0],[0],[0],[X0w[0,0]],[X0w[1,0]],[X0w[2,0]]])
    T = numpy.linspace(0, 12, 1000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        U2 = [0 for t in range(0,len(T))]
        U2[0] = u2
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        U2 = [u2 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        U2 = u2*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U2 = u2*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0, 0])
    # N.B. i primi 3 stati di xout sono quelli del sistema, mentre gli ultimi 3 sono quelli dell'osservatore
    
    fig = plt.figure(num='Simulation1', figsize=(16,17))
    mag, phase, omega = control.bode_plot(sys[0,0],Plot = False)
    mag = control.mag2db(mag)
    phase = phase*180/numpy.pi
    fig.add_subplot(321)
    plt.title('Bode plot: magnitude')
    plt.semilogx(omega,mag)
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('Mag. [dB]')
    plt.grid(True,which="both")
    
    fig.add_subplot(323)
    plt.semilogx(omega,phase)
    plt.title('Bode plot: phase')
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('Phase [deg]')
    plt.grid(True,which="both")
    
    fig.add_subplot(325)
    plt.title('Output response')
    plt.ylabel('Output')
    plt.plot(T,yout[0],T,yout_id,'g',T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$y$','$y_{ideal}$','Reference'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(322)
    plt.title('First state response')
    plt.ylabel('$x_1$')
    plt.plot(T,yout[3],T,yout[0],T,xout_id[0],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{1est}$','$x_{1real}$','$x_{1ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(324)
    plt.title('Second state response')
    plt.ylabel('$x_2$')
    plt.plot(T,yout[4],T,yout[1],T,xout_id[1],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{2est}$','$x_{2real}$','$x_{2ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(326)
    plt.title('Third state response')
    plt.ylabel('$x_3$')
    plt.plot(T,yout[5],T,yout[2],T,xout_id[2],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{3est}$','$x_{3real}$','$x_{3ideal}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Eigenvalues:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Eigenvalues:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          eig3o,
                                          widgets.Label(' ',border=3),
                                          widgets.VBox([widgets.Label('Inverse reference gain:',border=3),
                                                        widgets.Label('Inverse ideal reference gain:',border=3)]),
                                          widgets.VBox([gain_w2,gain_id_w2])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'DW':DW})
out2.layout.height = '1120px'
display(out2, alltogether2)

Output(layout=Layout(height='1120px'))